In [1]:
import wandb
import sys
import matplotlib.pyplot as plt
import scprep
import pandas as pd
sys.path.append('../src/')
from diffusion import DiffusionModel
from evaluate import get_results
from omegaconf import OmegaConf
from main import load_data, make_model
import numpy as np
import os
import glob
import matplotlib.animation as animation
from matplotlib.animation import PillowWriter
import torch

In [6]:
# Initialize wandb (replace 'your_entity' and 'your_project' with your specific details)
import torch
wandb.login()
api = wandb.Api()

# Specify your entity, project, and sweep ID
entity = "xingzhis"
project = "dmae"
sweep_id = 'n7krfbsz'

# Fetch the sweep
sweep = api.sweep(f"{entity}/{project}/{sweep_id}")

run_ids = [run.id for run in sweep.runs]
run_id = 'ucvq4x2o'
run = [run for run in sweep.runs if run.id == run_id][0]

cfg = OmegaConf.create(run.config)
folder_path = "../src/wandb/"
folder_list = glob.glob(f"{folder_path}*{run.id}*")
ckpt_files = glob.glob(f"{folder_list[0]}/files/*.ckpt")
ckpt_path = ckpt_files[0]
allloader, _, X, phate_coords, colors, dist, pp, _, _ = load_data(cfg, load_all=False)
emb_dim = phate_coords.shape[1]
data_path = os.path.join(cfg.data.root, cfg.data.name + cfg.data.filetype)
data = np.load(data_path, allow_pickle=True)
dist_std = np.std(data['dist'].flatten())
model = make_model(cfg, X.shape[1], emb_dim, pp, dist_std, from_checkpoint=True, checkpoint_path=ckpt_path)
model.eval()
x_all = next(iter(allloader))['x']
x_pred, z_pred = model(x_all)
x_pred = x_pred.detach().cpu().numpy()
z_pred = z_pred.detach().cpu().numpy()

TypeError: AEDist.__init__() missing 2 required positional arguments: 'dim' and 'emb_dim'

In [ ]:
# Initialize wandb (replace 'your_entity' and 'your_project' with your specific details)
wandb.login()
api = wandb.Api()

# Specify your entity, project, and sweep ID
entity = "xingzhis"
project = "dmae"
sweep_id = 'u05futg7'

# Fetch the sweep
sweep = api.sweep(f"{entity}/{project}/{sweep_id}")

run_ids = [run.id for run in sweep.runs]

run_id = 'xsj3odwl'
run = [run for run in sweep.runs if run.id == run_id][0]

In [ ]:
cfg = OmegaConf.create(run.config)
folder_path = "../src/wandb/"
folder_list = glob.glob(f"{folder_path}*{run.id}*")
ckpt_files = glob.glob(f"{folder_list[0]}/files/*.ckpt")
ckpt_path = ckpt_files[0]

In [ ]:
diffusionmodel = DiffusionModel.load_from_checkpoint(
    checkpoint_path=ckpt_path,
    data_size=10, 
    time_embedding_size=cfg.time_embedding_size,
    layer_widths=cfg.layer_widths,
    dropout=cfg.dropout,
    batch_norm=cfg.batch_norm,
    num_steps=cfg.num_steps,
    learning_rate=cfg.lr,
    weight_decay=cfg.weight_decay,
)

/gpfs/gibbs/pi/krishnaswamy_smita/xingzhi/.conda_envs/geosink/lib/python3.11/site-packages/pytorch_lightning/utilities/parsing.py:269: UserWarning: Attribute 'activation_fn' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['activation_fn'])`.
  rank_zero_warn(


In [ ]:
train_mask = data['is_train']
data_train = data['data'][train_mask]
color_train = data['colors'][train_mask]

In [ ]:
data_pred = x_pred[~train_mask]
latent_pred = z_pred[~train_mask]
with torch.no_grad():
    generated_latent = diffusionmodel.generate_samples((~train_mask).sum())

In [ ]:
data_latent = np.load(cfg.path + cfg.data)
data_latent_train = data_latent['data'][data_latent['train_mask']]
mean_, std_ = np.mean(data_latent_train, axis=0), np.std(data_latent_train, axis=0)

In [ ]:
generated_latent = generated_latent * std_ + mean_

In [ ]:
with torch.no_grad():
    generated_ambient = model.decode(generated_latent)

In [ ]:
generated = generated_ambient.detach().cpu().numpy()

In [ ]:
import numpy as np
import matplotlib.pylab as pl
import ot
import ot.plot

In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
latent_gen = generated_latent.detach().cpu().numpy()

In [ ]:
latent_test = data_latent['data'][~data_latent['train_mask']]

In [ ]:
M = ot.dist(latent_gen, latent_test)
M = torch.tensor(M, dtype=torch.float32, device=device)
M/=M.max()

In [ ]:
n = latent_test.shape[0]
a, b = torch.ones((n,)).to(device) / n, torch.ones((n,)).to(device) / n  # uniform distribution on samples

In [ ]:
G0 = ot.emd(a, b, M)

/gpfs/gibbs/pi/krishnaswamy_smita/xingzhi/.conda_envs/geosink/lib/python3.11/site-packages/ot/lp/__init__.py:354: UserWarning: numItermax reached before optimality. Try to increase numItermax.
  result_code_string = check_result(result_code)


In [ ]:
W = torch.sum(G0 * M)

In [ ]:
W

tensor(1733.2749, device='cuda:0')